In [15]:
%%pyspark
df = spark.read.load('abfss://files@datalakeqfpnae6.dfs.core.windows.net/products/products.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(sparkqfpnae6, 2, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2f103c39-3dcd-4311-9fb8-996fa4597165)

In [16]:
 delta_table_path = "/delta/products-delta"
 df.write.format("delta").save(delta_table_path)

StatementMeta(sparkqfpnae6, 2, 22, Finished, Available, Finished)

AnalysisException: Cannot write to already existent path abfss://files@datalakeqfpnae6.dfs.core.windows.net/delta/products-delta without setting OVERWRITE = 'true'.

In [17]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)

StatementMeta(sparkqfpnae6, 2, 23, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|2753.9919|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [18]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)

StatementMeta(sparkqfpnae6, 2, 24, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [19]:
 deltaTable.history(10).show(20, False, True)

StatementMeta(sparkqfpnae6, 2, 25, Finished, Available, Finished)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 2                                                                                                                                                                                                                                    
 timestamp           | 2025-02-24 16:45:25.761                                                                                                                                                                                                              
 userId              | null                                                                                                                                                                                                                      

In [20]:
 spark.sql("CREATE DATABASE AdventureWorks")
 spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

StatementMeta(sparkqfpnae6, 2, 26, Finished, Available, Finished)

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `AdventureWorks` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [21]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsExternal;

StatementMeta(, 2, 28, Finished, , Finished)

<Spark SQL result set with 0 rows and 0 fields>

Error: [TABLE_OR_VIEW_NOT_FOUND] The table or view `ProductsExternal` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 15;
'Project [*]
+- 'UnresolvedRelation [ProductsExternal], [], false


In [22]:
 df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

StatementMeta(sparkqfpnae6, 2, 29, Finished, Available, Finished)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                       |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                          |null   |
|ProductName                 |string                                                                                                                          |null   |
|Category                    |string                                                                                                                          |n

In [23]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsManaged;

StatementMeta(sparkqfpnae6, 2, 31, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [24]:
 %%sql

 USE AdventureWorks;

 DROP TABLE IF EXISTS ProductsExternal;
 DROP TABLE IF EXISTS ProductsManaged;

StatementMeta(sparkqfpnae6, 2, 34, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [25]:
 %%sql

 USE AdventureWorks;

 CREATE TABLE Products
 USING DELTA
 LOCATION '/delta/products-delta';

StatementMeta(sparkqfpnae6, 2, 36, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [26]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM Products;

StatementMeta(sparkqfpnae6, 2, 38, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [27]:
 from notebookutils import mssparkutils
 from pyspark.sql.types import *
 from pyspark.sql.functions import *

 # Create a folder
 inputPath = '/data/'
 mssparkutils.fs.mkdirs(inputPath)

 # Create a stream that reads data from the folder, using a JSON schema
 jsonSchema = StructType([
 StructField("device", StringType(), False),
 StructField("status", StringType(), False)
 ])
 iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

 # Write some event data to the folder
 device_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''
 mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
 print("Source stream created...")

StatementMeta(sparkqfpnae6, 2, 39, Finished, Available, Finished)

Source stream created...


In [28]:
 # Write the stream to a delta table
 delta_stream_table_path = '/delta/iotdevicedata'
 checkpointpath = '/delta/checkpoint'
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")

StatementMeta(sparkqfpnae6, 2, 40, Finished, Available, Finished)

Streaming to delta sink...


In [29]:
 # Read the data in delta format into a dataframe
 df = spark.read.format("delta").load(delta_stream_table_path)
 display(df)

StatementMeta(sparkqfpnae6, 2, 41, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9831abb4-bad7-4ab9-af78-a2682350466b)

In [30]:
 # create a catalog table based on the streaming sink
 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

StatementMeta(sparkqfpnae6, 2, 42, Finished, Available, Finished)

DataFrame[]

In [31]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparkqfpnae6, 2, 43, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [32]:
 # Add more data to the source stream
 more_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(sparkqfpnae6, 2, 44, Finished, Available, Finished)

True

In [33]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparkqfpnae6, 2, 45, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [34]:
 deltastream.stop()

StatementMeta(sparkqfpnae6, 2, 46, Finished, Available, Finished)